In [152]:
import os
from glob import glob

import dask
import dask.bag as db
import dask.dataframe as dd
from distributed import Client
from dask_jobqueue import SLURMCluster
import pandas as pd 
import numpy as np

from IPython.display import display
import matplotlib.pyplot as plt

import sqlite3

In [153]:
# Set LOCAL to True for single-machine execution while developing
# Set LOCAL to False for cluster execution
LOCAL = False

if LOCAL:
    # This line creates a single-machine dask client
    client = Client()
else:    
    # This line creates a SLURM cluster dask and dask client
    # Logging outputs will be stored in /scratch/{your-netid}
    
    cluster = SLURMCluster(memory='4GB', cores=2, python='/scratch/work/public/dask/bin/python', 
                               local_directory='/tmp/{}/'.format(os.environ['SLURM_JOB_USER']),
                               job_extra=['--output=/scratch/{}/slurm-%j.out'.format(os.environ['SLURM_JOB_USER'])])

    cluster.submit_command = 'slurm'
    cluster.scale(100)

    display(cluster)
    client = Client(cluster)

display(client)

/ext3/miniconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37317 instead
  warnings.warn(


Client Scheduler: tcp://10.32.35.14:37099 Dashboard: http://10.32.35.14:37317/status,Cluster Workers: 58 Cores: 58 Memory: 116.00 GB


In [154]:
public_path = '/scratch/work/public/MillionSongDataset/AdditionalFiles/'

In [22]:
os.listdir(public_path)

['BROKEN_lastfm_tags.db',
 'README',
 'lastfm_tags.db',
 'LICENSE',
 'unique_terms.txt',
 'artist_term.db',
 'artist_location.txt',
 'artist_similarity.db',
 'mxm_779k_matches.txt',
 'sid_mismatches.csv',
 'rdio_ids.txt',
 'sid_mismatches.txt',
 'TRACKIDS.txt',
 'msd_summary_file.h5',
 'unique_tracks.txt',
 'mxm_dataset.db',
 'unique_mbtags.txt',
 'unique_artists.txt',
 'tracks_per_year.txt',
 'track_metadata.db']

In [8]:
# conn = sqlite3.connect(path + '.db')
# cur = conn.cursor()
# cur.execute("SELECT * FROM artist_term LIMIT 1")
# cur.execute("""SELECT sql FROM sqlite_master WHERE type='table'""")
# print(cur.fetchall())

# Save tables to local folder

In [9]:
conn = sqlite3.connect(public_path + 'track_metadata.db')
df = pd.read_sql_query("SELECT * from songs", conn).replace([None], np.nan) 
# df.to_csv('/scratch/tj810/final-project-team_unsupervised_learners/feature_files/track_metadata.csv',index=False)

In [138]:
conn = sqlite3.connect(public_path + 'artist_term.db')
df = pd.read_sql_query("SELECT * from artist_term", conn)\
        .replace([None], np.nan)\
        .groupby('artist_id')['term']\ # combine all terms associated with each artist
        .apply(list).reset_index()
# df.to_csv('/scratch/tj810/final-project-team_unsupervised_learners/feature_files/artist_term.csv',index=False)

# Combine all features

In [155]:
local_path = '/scratch/tj810/final-project-team_unsupervised_learners/feature_files/'

In [156]:
# features: duration, artist_familiarity, artist_hotttnessss, year
# includes track_id, artist_id
track_metadata = dd.read_csv(local_path + 'track_metadata.csv', header = 0).repartition(100)

# features: artist location (map using artist_id)
artist_location = dd.read_csv(public_path+'artist_location.txt',
                              sep= '<SEP>',
                              header = None, 
                              names = ['artist_id','latitude','longtitude','artist_name','location'])

# features: artist term (map using artist_id)
artist_term = dd.read_csv(local_path + 'artist_term.csv', header = 0)

/ext3/miniconda3/lib/python3.8/site-packages/dask/dataframe/io/csv.py:538: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  head = reader(BytesIO(b_sample), **kwargs)


In [157]:
print(artist_term.columns) # (43943, 2)
print(artist_location.columns) # 13850 rows × 5 columns
print(track_metadata.columns) # (1000000, 11)

Index(['artist_id', 'term'], dtype='object')
Index(['artist_id', 'latitude', 'longtitude', 'artist_name', 'location'], dtype='object')
Index(['track_id', 'title', 'song_id', 'release', 'artist_id', 'artist_mbid',
       'artist_name', 'duration', 'artist_familiarity', 'artist_hotttnesss',
       'year'],
      dtype='object')


In [158]:
artist_features = artist_term.merge(artist_location, how='outer', on = 'artist_id').repartition(npartitions=1)
all_features = track_metadata.merge(artist_features, how='left', on = 'artist_id').compute()

In [167]:
# all_features.drop(columns='artist_name_y').rename(columns={'artist_name_x':'artist_name'}).reset_index(drop=True).to_csv(local_path + 'all_features.csv')

In [196]:
features.loc[2]

track_id                                             TRMMMRX128F93187D9
title                                                 No One Could Ever
song_id                                              SOGTUKN12AB017F4F1
release                                                          Butter
artist_id                                            ARGEKB01187FB50750
artist_mbid                        3d403d44-36ce-465c-ad43-ae877e65adc4
artist_name                                              Hudson Mohawke
duration                                                      138.97098
artist_familiarity                                             0.643681
artist_hotttnesss                                              0.437504
year                                                               2006
term                  ['broken beat', 'hip hop', 'trip hop', 'glitch...
latitude                                                        55.8578
longtitude                                                     -

In [203]:
# for i in range(features.shape[1]):
#     print(f'Number of unique {features.columns[i]}:')
#     print(features[features.columns[i]].unique().size)

Number of unique track_id:
1000000
Number of unique title:
702429
Number of unique song_id:
999056
Number of unique release:
149289
Number of unique artist_id:
44745
Number of unique artist_mbid:
38730
Number of unique artist_name:
72665
Number of unique duration:
32145
Number of unique artist_familiarity:
47218
Number of unique artist_hotttnesss:
43476
Number of unique year:
90
Number of unique term:
43152
Number of unique latitude:
2640
Number of unique longtitude:
2643
Number of unique location:
3990


In [ ]:
song_hotttnesss
loudness
mode
tempo
time_signature
time_signature_confidence

In [ ]:
def get_artistid_trackid_artistname(trackfile):
    """
    Utility function, opens a h5 file, gets the 4 following fields:
     - artist Echo Nest ID
     - artist Musicbrainz ID
     - track Echo Nest ID
     - artist name
    It is returns as a triple (,,)
    Assumes one song per file only!
    """
    h5 = hdf5_utils.open_h5_file_read(trackfile)
    assert GETTERS.get_num_songs(h5) == 1,'code must be modified if more than one song per .h5 file'
    aid = GETTERS.get_artist_id(h5)
    ambid = GETTERS.get_artist_mbid(h5)
    tid = GETTERS.get_track_id(h5)
    aname = GETTERS.get_artist_name(h5)
    h5.close()
    return aid,ambid,tid,aname